In [2]:
import numpy as np
import pandas as pd
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing
data = pd.read_csv(r'/home/supriya/Hybrid_v2_v2x.csv',dtype={'Error_Label':object} )
print(data.shape)
data.head()

(67677, 1401)


,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,timestamp gasprice,timestamp number,timestamp origin,timestamp return,timestamp revert,timestamp selfdestruct,timestamp sha3,timestamp sub,timestamp timestamp,Error_Label
0,10001,0.41463,1.18500,1.12590,-0.63784,-1.6094,-0.19869,-0.23010,-0.69190,-0.997910,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,000000000000000100
1,10005,0.32088,-1.08450,-1.15780,-0.63845,1.0306,0.38707,0.95113,0.30132,-1.087500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,000000000001000000
2,10007,-1.21220,0.54410,0.33130,1.34610,-0.0440,-0.68850,-2.00540,0.73690,0.560400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019261,010000000000001000
3,10010,-0.94921,0.58352,-0.38527,-1.02860,-1.6585,0.23852,-1.19810,-0.40317,0.067658,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,000000010000000000
4,10011,0.05630,1.40680,0.39810,0.29180,-0.2699,-0.66170,-1.55760,-0.99460,0.196800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010843,001001000001000000


In [4]:
# split labels and return new dataframe
#Create independent and Dependent Features
ERROR_LABEL=6
def split_label_dict(df):
    # Change 'label' to whatever naming used in the original DataFrame.
    temp_df = df['Error_Label'].str.split("", n = -1, expand = True)
    for i in range(1, 19):
        df[f'l{i}'] = temp_df[i]
        df[f'l{i}'] = pd.to_numeric(df[f'l{i}'])
    return df

newdata=split_label_dict(data)
#print(newdata)
#Create independent and Dependent Features
df = pd.DataFrame(data)
new_df = df.drop(['Error_Label','l18','l17','l16','l15','l14','l13','l12','l11','l10','l9','l8','l7','l6','l5','l3','l4','l2','l1', 'ID'],axis=1)
#new_df = data.drop(['Error_Label','ID'],axis=1)
X = new_df
# Store the variable we are predicting 
Y = newdata[f"l{ERROR_LABEL}"]

# Print the shapes of X & Y
print(X.shape,Y.shape)
print(X)
#print(Y)

(67677, 1399) (67677,)
            v1       v2       v3       v4        v5       v6        v7  \
0      0.41463  1.18500  1.12590 -0.63784 -1.609400 -0.19869 -0.230100   
1      0.32088 -1.08450 -1.15780 -0.63845  1.030600  0.38707  0.951130   
2     -1.21220  0.54410  0.33130  1.34610 -0.044000 -0.68850 -2.005400   
3     -0.94921  0.58352 -0.38527 -1.02860 -1.658500  0.23852 -1.198100   
4      0.05630  1.40680  0.39810  0.29180 -0.269900 -0.66170 -1.557600   
...        ...      ...      ...      ...       ...      ...       ...   
67672  0.57810 -3.40390  1.62580 -0.33680  2.292200 -1.02060 -0.836500   
67673 -2.12190 -2.21790 -2.69700  2.89430  1.364600  1.37550  0.255070   
67674 -1.60550  0.50686 -0.49615 -0.52569 -0.071134 -1.77820  0.008866   
67675 -1.04400 -1.67610  0.18878  2.53450  0.071368  0.99836 -0.079039   
67676  1.83510  3.37730  1.35650 -0.69700 -1.715700 -2.39180 -0.266600   

            v8        v9      v10  ...  timestamp gaslimit  \
0     -0.69190 -0.997910  